# Preprocessing

In [ ]:
import pandas as pd
import numpy as np

class SolarDataProcessor:
    """
    A class for processing and combining solar inverter data with weather data.

    This class handles the preprocessing of inverter data by splitting it by inverter,
    filling missing values, and combining with weather data.
    """

    def __init__(self):
        """Initialize the SolarDataProcessor."""
        self.df = None
        self.weather_df = None
        self.combined_df = None

    def read_data(self, inverter_csv_path, weather_csv_path):
        """
        Read the inverter and weather data from CSV files.

        Parameters:
        inverter_csv_path (str): Path to the inverter CSV file
        weather_csv_path (str): Path to the weather CSV file
        """
        self.df = pd.read_csv(inverter_csv_path)
        self.weather_df = pd.read_csv(weather_csv_path)

        # Set pandas display options
        pd.set_option('display.max_columns', None)

        return self

    def split_inverter_data(self):
        """
        Split the inverter data by inverter number (1-3) and clean column names.
        """
        # Define additional columns to include in each inverter DataFrame
        additional_columns = ["Date", "Time"]

        # Initialize dictionary to store inverter columns
        inverter_columns = {1: [], 2: [], 3: []}

        # Identify columns for each inverter
        for col in self.df.columns:
            if "INVERTER 1" in col:
                inverter_columns[1].append(col)
            elif "INVERTER 2" in col:
                inverter_columns[2].append(col)
            elif "INVERTER 3" in col:
                inverter_columns[3].append(col)

        # Create DataFrames for each inverter and rename columns
        inverter_dfs = {}
        for inverter, cols in inverter_columns.items():
            # Ensure additional columns are included
            cols_with_additional = additional_columns + cols

            # Clean column names by removing the inverter identifier
            clean_cols = [col.replace(f"INVERTER {inverter}", "") for col in cols]

            # Keep the original names for additional columns
            clean_cols = additional_columns + clean_cols

            # Create a DataFrame for each inverter
            inverter_dfs[inverter] = self.df[cols_with_additional].copy()
            inverter_dfs[inverter].columns = clean_cols
            inverter_dfs[inverter].columns = inverter_dfs[inverter].columns.str.strip()

        one = inverter_dfs[1]
        two = inverter_dfs[2]
        three = inverter_dfs[3]

        return one, two, three

    def fill_missing_data(self, one, two, three):
        """
        Fill short gaps in solar inverter data using linear interpolation.
        """
        # Define the fill_missing_solar_data function
        def fill_missing_solar_data(df, columns):
            """
            Fill short gaps in solar inverter data using linear interpolation

            Parameters:
            df: DataFrame with datetime index
            columns: List of columns to process
            """
            df_filled = df.copy()

            for column in columns:
                if column not in df.columns:
                    print(f"Column '{column}' not found in DataFrame.")
                    continue

                # Simple linear interpolation for gaps up to 4 points
                df_filled[column] = df_filled[column].interpolate(
                    method='linear',
                    limit=4  # Only fill gaps of 4 or fewer points
                )
            return df_filled

        def interpolate_outliers(df, columns, factor=1.5):
          """
          Detect and replace outliers using IQR and linear interpolation.

          Parameters:
          - df: pd.DataFrame — Input DataFrame.
          - columns: list — Columns to check for outliers.
          - factor: float — IQR multiplier for determining outlier thresholds (default=1.5).

          Returns:
          - pd.DataFrame — DataFrame with outliers replaced via interpolation.
          """
          df_clean = df.copy()

          for col in columns:
              if col in ["Date", "Time"]:
                  continue

              Q1 = df_clean[col].quantile(0.25)
              Q3 = df_clean[col].quantile(0.75)
              IQR = Q3 - Q1
              lower_bound = Q1 - factor * IQR
              upper_bound = Q3 + factor * IQR

              # Mark outliers as NaN
              mask_outliers = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
              df_clean.loc[mask_outliers, col] = np.nan

              # Interpolate to fill in outliers
              df_clean[col] = df_clean[col].interpolate(limit_direction='both')

          return df_clean


        cols1 = [col for col in one.columns if col not in ["Date", "Time"]]
        cols2 = [col for col in two.columns if col not in ["Date", "Time"]]
        cols3 = [col for col in three.columns if col not in ["Date", "Time"]]

        one = interpolate_outliers(one, cols1)
        two = interpolate_outliers(two, cols2)
        three = interpolate_outliers(three, cols3)

        one = fill_missing_solar_data(one, cols1)
        two = fill_missing_solar_data(two, cols2)
        three = fill_missing_solar_data(three, cols3)

        return one, two, three

    def combine_inverter_dfs(self, one, two, three):
        """
        Combine the three inverter DataFrames into a single DataFrame.
        """
        # Rename columns to include inverter number suffixes
        one = one.rename(columns={col: f"{col}_inv1" for col in one.columns if col not in ["Date", "Time"]})
        two = two.rename(columns={col: f"{col}_inv2" for col in two.columns if col not in ["Date", "Time"]})
        three = three.rename(columns={col: f"{col}_inv3" for col in three.columns if col not in ["Date", "Time"]})

        # Merge the three DataFrames on "Date" and "Time"
        df = one.merge(two, on=["Date", "Time"]).merge(three, on=["Date", "Time"])

        # Identify unique metric types by extracting column names without inverter numbers
        metric_types = set(col.rsplit("_", 1)[0] for col in df.columns if col not in ["Date", "Time"])

        # Aggregate metrics
        for metric in metric_types:
            # Select all inverter columns for this metric based on suffixes
            columns_to_combine = [col for col in df.columns if col.startswith(metric) and col not in ["Date", "Time"]]

            # Combine the columns (sum or mean, depending on metric type)
            if any(word in metric.lower() for word in ["power", "energy", "connections"]):
                df[f"total_{metric}"] = df[columns_to_combine].sum(axis=1)
            elif any(word in metric.lower() for word in ["temperature", "average", "percent"]):
                df[f"average_{metric}"] = df[columns_to_combine].mean(axis=1)
            else:
                df[f"total_{metric}"] = df[columns_to_combine].sum(axis=1)

            # Drop the original columns for this metric
            df.drop(columns=columns_to_combine, inplace=True)

        self.combined_df = df
        return self


    def add_datetime_and_hour(self):
        self.combined_df['Datetime'] = pd.to_datetime(self.combined_df['Date'] + ' ' + self.combined_df['Time'])
        self.combined_df['hour'] = self.combined_df['Datetime'].dt.hour

        self.weather_df['Datetime'] = pd.to_datetime(self.weather_df['Date'] + ' ' + self.weather_df['Time'])
        self.weather_df['hour'] = self.weather_df['Datetime'].dt.hour

        self.combined_df = self.combined_df.sort_values('Datetime')
        self.weather_df = self.weather_df.sort_values('Datetime')

        return self

    def merge_weather_data(self):
        self.combined_df = pd.merge_asof(self.combined_df, self.weather_df, on='Datetime', direction='nearest')

        self.combined_df = self.combined_df.drop(columns=['hour_x', 'Date_y', 'Time_y'])
        self.combined_df = self.combined_df.rename(columns={
            'Date_x':'date',
            'Time_x':'time',
            'hour_y':'hour'
        })
        return self

    def resample_time_series(self, data, interval='10min', tolerance='5min'):
        """
        Resample a time series to a consistent interval using closest real values when available,
        otherwise interpolate.

        Parameters:
        -----------
        data : pandas.DataFrame
            Original time series data with a 'Datetime' column or 'date' and 'time' columns.
        interval : str
            Desired resampling interval (e.g., '10min').
        tolerance : str
            Time window to search for a nearby real value (e.g., '2min').

        Returns:
        --------
        pandas.DataFrame
            Resampled DataFrame with consistent intervals using closest available data or interpolation.
        """

        df = data.copy()

        if 'Datetime' in df.columns:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
        else:
            df['Datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

        df.set_index('Datetime', inplace=True)
        df.sort_index(inplace=True)

        original_index = df.index
        resample_index = pd.date_range(start=original_index.min(), end=original_index.max(), freq=interval)
        tolerance_offset = pd.to_timedelta(tolerance)

        # Initialize resampled DataFrame
        df_resampled = pd.DataFrame(index=resample_index)

        numeric_cols = df.select_dtypes(include='number').columns.tolist()

        for col in numeric_cols:
            resampled_col = []

            for target_time in resample_index:
                # Find the closest existing time within tolerance
                mask = (original_index >= target_time - tolerance_offset) & (original_index <= target_time + tolerance_offset)
                candidates = df.loc[mask, col]

                if not candidates.empty:
                    # Use the value closest in time
                    closest_time = candidates.index[np.argmin(np.abs(candidates.index - target_time))]
                    resampled_col.append(df.loc[closest_time, col])
                else:
                    resampled_col.append(np.nan)

            df_resampled[col] = resampled_col

            # Interpolate any remaining missing values
            df_resampled[col] = df_resampled[col].interpolate(method='time', limit_direction='both')

        # Final fill if needed (still NaNs after interpolation)
        df_resampled = df_resampled.ffill().bfill()

        # Summary
        print(f"\nResampled to {interval} with tolerance {tolerance}")
        print(f"Resulting rows: {len(df_resampled)}")
        remaining_nans = df_resampled.isna().sum()
        if remaining_nans.sum() > 0:
            print("Warning: Columns with remaining NaNs:")
            print(remaining_nans[remaining_nans > 0])
        else:
            print("All missing values successfully filled.")

        self.combined_df = df_resampled
        return self


    def process_data(self, inverter_csv_path, weather_csv_path):
        """
        Process the data from start to finish in a single method call.
        ****** If you want to use the resample_time_series method, you will have to add to this method. I added it so we can get the timestamps at the same resolution
            it automatically updates the combined_df so all you should have to add to this method is self.resample_time_series()

        """
        self.read_data(inverter_csv_path, weather_csv_path)
        one, two, three = self.split_inverter_data()
        one, two, three = self.fill_missing_data(one, two, three)
        self.combine_inverter_dfs(one, two, three)
        self.add_datetime_and_hour()
        self.merge_weather_data()
        self.resample_time_series(data=self.combined_df)

        return self.combined_df  # Ensure you return the actual processed DataFrame

# example usage
processor = SolarDataProcessor()
df = processor.process_data('Inv_2024-09-30.csv', 'weather_2024-09-30.csv')

# LSTM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
class LSTMForecastModel:
    def __init__(self, data=None, file_path=None, lookback=5): # lookback is how many rows back in the data the model is seeing. you will need to adjust this if you
                                                              # adjsut how you resample the data (I commented on this below in the preprocess data method)
        """
        Initialize the LSTM model with either a DataFrame or a CSV file path.
        Parameters:
        - data: pandas DataFrame containing the input data.
        - file_path: path to the CSV file containing the data.
        - lookback: Number of past timesteps to use as input for LSTM.
        """
        if data is not None:
            self.df = data.copy()
        elif file_path is not None:
            self.df = pd.read_csv(file_path)
        else:
            raise ValueError("Either a DataFrame or a file path must be provided.")
        self.lookback = lookback
        self.scaler = MinMaxScaler()
        self.features = [
            'historical_power','average_temperature_C', 'WMS 01 irradiance', 'hours_since_sunrise','hours_until_sunset','peak_sun_factor','is_daytime',
            'total_last_active_fault', 'total_net_AC_voltage_V','total_time_run_today_h', 'average_average_cosphii_percent',
       'total_DC_voltage_DCV', 'total_output_current_A',
       'total_grid_connections','total_net_frequency_Hz'
        ]# is_daytime

        # Define what features to engineer and how
        self.feature_engineering_plan = {
            'total_output_power_kW': {
                'lags': [1, 2, 3, 6, 12, 24],
                'rolling': {'mean': [1,3, 6, 12,24], 'std': [3,6], 'min': [3,6], 'max': [1,2,3,6]},
                'diff': [1, 2,3,4,5]
            },
            'average_temperature_C': {
                'lags': [1, 3, 6, 12],
                'rolling': {'mean': [1,2,3, 6,12,24], 'std': [2,3,6]},
                'diff': [1, 2,3]
            },
            'total_DC_voltage_DCV': {
                'lags': [1, 3, 6],
                'rolling': {'mean': [1,2,3, 6,12,24], 'std': [2,3,6]},
                'diff': [1, 2,3]
            },
            'total_output_current_A': {
                'lags': [1, 3, 6],
                'rolling': {'mean': [1,2,3, 6,12,24], 'std': [2,3,6]},
                'diff': [1, 2,3]
            },
            'average_average_cosphii_percent': {
                'lags': [1, 3, 6],
                'rolling': {'mean': [1,2,3, 6,12,24]},
                'diff': [1, 2,3]
            },
            'total_time_run_today_h': {
                'lags': [1, 3, 6],
                'rolling': {'sum': [1,3, 6,12,24]},
                'diff': [1, 2,3]
            },
            'total_last_active_fault': {
                'lags': [1, 3],
                'rolling': {'sum': [1,3, 6,12,24]},
                'diff': [1, 2,3]
            }
        }

        if 'WMS 01 irradiance' in self.df.columns:
            self.feature_engineering_plan['WMS 01 irradiance'] = {
                'lags': [1, 2, 3, 6, 12, 24],
                'rolling': {'mean': [1,3, 6, 12], 'std': [2,3,6], 'max': [1,2, 3,6]},
                'diff': [1, 2, 3,4,5]
            }
        self.target = None

        '''
       for self.features you can add more columns to this if you want. but these are some of the ones that i have been using.
        '''

    def preprocess_data(self, resample_freq='30min', target_horizon=18):
        """
        Preprocess the dataset for LSTM input using XGBoost-style resampling.
        - resample_freq: frequency string (e.g., '10min', '30min')
        - target_horizon: number of rows (at original freq) to shift target forward
        """
        if 'datetime' in self.df.columns:
            self.df['datetime'] = pd.to_datetime(self.df['datetime'])
            self.df.set_index('datetime', inplace=True)
        elif 'index' in self.df.columns:
            self.df['datetime'] = pd.to_datetime(self.df['index'])
            self.df.set_index('datetime', inplace=True)

        self.df['hour'] = self.df.index.hour
        self.df['day'] = self.df.index.day
        self.df['month'] = self.df.index.month
        self.df['dayofweek'] = self.df.index.dayofweek
        self.df['quarter'] = self.df.index.quarter
        self.df['minute'] = self.df.index.minute

        self.df['hours_since_sunrise'] = (self.df['hour'] + self.df['minute'] / 60 - 6).clip(lower=0)
        self.df['hours_until_sunset'] = (18 - (self.df['hour'] + self.df['minute'] / 60)).clip(lower=0)
        self.df['is_daytime'] = ((self.df['hour'] >= 6) & (self.df['hour'] < 18)).astype(int)

        self.df['peak_sun_factor'] = np.sin(np.pi * (self.df['hour'] + self.df['minute']/60 - 6) / 12)
        self.df['peak_sun_factor'] = self.df['peak_sun_factor'].clip(lower=0)

        self.df['hour_sin'] = np.sin(2 * np.pi * self.df['hour'] / 24)
        self.df['hour_cos'] = np.cos(2 * np.pi * self.df['hour'] / 24)
        self.df['month_sin'] = np.sin(2 * np.pi * self.df['month'] / 12)
        self.df['month_cos'] = np.cos(2 * np.pi * self.df['month'] / 12)

        # Original target
        original_power = self.df['total_output_power_kW'].copy()

        # Resample non-target features
        df_features = self.df.drop(columns=['total_output_power_kW'])
        df_resampled = df_features.select_dtypes(include=[np.number]).resample(resample_freq).mean()

        # Resample power for input (to include historical context)
        df_resampled['historical_power'] = original_power.resample(resample_freq).mean()

        # Shift original target to create a future prediction target
        future_target = original_power.shift(-target_horizon)
        df_resampled['power_future'] = future_target.resample(resample_freq).first()

        # Align future target to df_resampled
        common_idx = df_resampled.index.intersection(future_target.index)
        df_resampled = df_resampled.loc[common_idx].copy()
        df_resampled['power_future'] = future_target.loc[common_idx]

        # Feature engineering
        for col, ops in self.feature_engineering_plan.items():
            if col not in df_resampled.columns:
                print(f"Warning: {col} not found in dataframe, skipping")
                continue

            if 'lags' in ops:
                for lag in ops['lags']:
                    feature_name = f'{col}_lag_{lag}'
                    df_resampled[feature_name] = df_resampled[col].shift(lag)
                    self.features.append(feature_name)

            if 'rolling' in ops:
                for method, windows in ops['rolling'].items():
                    for window in windows:
                        feature_name = f'{col}_{method}_{window}'
                        if method == 'mean':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).mean()
                        elif method == 'sum':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).sum()
                        elif method == 'std':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).std()
                        elif method == 'min':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).min()
                        elif method == 'max':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).max()
                        self.features.append(feature_name)

            if 'diff' in ops:
                for periods in ops['diff']:
                    feature_name = f'{col}_diff_{periods}'
                    df_resampled[feature_name] = df_resampled[col].diff(periods=periods)
                    self.features.append(feature_name)

        self.df_clean = df_resampled.dropna()

        # Scale input features only
        self.df_clean[self.features] = self.scaler.fit_transform(self.df_clean[self.features])

        # add for later use in blender
        self.target = self.df_clean['power_future']

    def create_sequences(self):
        """Convert the dataframe into sequences for LSTM input."""
        X, y = [], []
        data = self.df_clean[self.features].values
        target = self.df_clean['power_future'].values
        for i in range(len(data) - self.lookback):
            X.append(data[i:i + self.lookback])
            y.append(target[i + self.lookback])
        self.X = np.array(X)
        self.y = np.array(y)

        # Train-test split (80% train, 20% test)
        split_idx = int(len(self.X) * 0.8)
        self.X_train, self.X_test = self.X[:split_idx], self.X[split_idx:]
        self.y_train, self.y_test = self.y[:split_idx], self.y[split_idx:]

    def build_model(self):
        """Define the LSTM model architecture."""
        model = Sequential([
            LSTM(128, activation='tanh', return_sequences=True, input_shape=(self.lookback, len(self.features))),
            Dropout(0.2),
            LSTM(64, activation='tanh'),
            Dropout(0.2),
            Dense(50, activation='relu'),
            Dense(25, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
        self.model = model

    def train_model(self, epochs=50, batch_size=16):
        """Train the LSTM model and store history for plotting."""
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        self.history = self.model.fit(
            self.X_train, self.y_train, epochs=epochs, batch_size=batch_size,
            validation_data=(self.X_test, self.y_test), verbose=1, callbacks=[early_stopping]
        )

    def evaluate_model(self):
        """Evaluate the LSTM model on the test set."""
        y_pred = self.model.predict(self.X_test)
        y_pred = np.clip(y_pred, a_min=0, a_max=None)  # for non-negative predictions
        self.metrics = {
            'rmse': np.sqrt(mean_squared_error(self.y_test, y_pred)),
            'mae': mean_absolute_error(self.y_test, y_pred),
            'r2': r2_score(self.y_test, y_pred)
        }
        return self.metrics

    def plot_results(self):
        """Plot actual vs predicted power output."""
        y_pred = self.model.predict(self.X_test)
        y_pred = np.clip(y_pred, a_min=0, a_max=None)
        plt.figure(figsize=(12, 6))
        plt.plot(self.y_test, label='Actual', alpha=0.8)
        plt.plot(y_pred, label='Predicted', alpha=0.8)
        plt.title('Hourly Solar Power Prediction (LSTM, 3 hours ahead)')
        plt.ylabel('Power Output (kW)')
        plt.xlabel('Time')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

    def plot_training_history(self):
        """Plot training & validation loss and MAE."""
        fig, ax1 = plt.subplots(figsize=(12, 6))
        # Plot loss
        ax1.plot(self.history.history['loss'], label='Train Loss', color='blue')
        ax1.plot(self.history.history['val_loss'], label='Validation Loss', color='blue', linestyle='dashed')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Loss (MSE)')
        ax1.set_title('Training & Validation Loss')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        plt.show()
        # Plot MAE
        fig, ax2 = plt.subplots(figsize=(12, 6))
        ax2.plot(self.history.history['mae'], label='Train MAE', color='red')
        ax2.plot(self.history.history['val_mae'], label='Validation MAE', color='red', linestyle='dashed')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('MAE')
        ax2.set_title('Training & Validation MAE')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        plt.show()

    def predict(self, new_data, resample_freq='30min', target_horizon=18):
        """
        Generate predictions on new, preprocessed data.

        Parameters:
        - new_data: pandas DataFrame with original structure (before resampling).
        - resample_freq: resampling frequency to match training.
        - target_horizon: number of future steps predicted ahead.

        Returns:
        - predictions: pandas Series with timestamps aligned.
        """
        if self.model is None:
            raise ValueError("Model not built or trained yet. Call train_model() first.")
        if not hasattr(self, 'features') or not self.features:
            raise ValueError("Feature list not found. Ensure preprocess_data() was called before training.")

        # Store original features to avoid modifying them
        original_features = self.features.copy()

        # Clone and preprocess new data
        temp_df = new_data.copy()

        if 'datetime' in temp_df.columns:
            temp_df['datetime'] = pd.to_datetime(temp_df['datetime'])
            temp_df.set_index('datetime', inplace=True)
        elif 'index' in temp_df.columns:
            temp_df['datetime'] = pd.to_datetime(temp_df['index'])
            temp_df.set_index('datetime', inplace=True)

        temp_df['hour'] = temp_df.index.hour
        temp_df['day'] = temp_df.index.day
        temp_df['month'] = temp_df.index.month
        temp_df['dayofweek'] = temp_df.index.dayofweek
        temp_df['quarter'] = temp_df.index.quarter
        temp_df['minute'] = temp_df.index.minute

        temp_df['hours_since_sunrise'] = (temp_df['hour'] + temp_df['minute'] / 60 - 6).clip(lower=0)
        temp_df['hours_until_sunset'] = (18 - (temp_df['hour'] + temp_df['minute'] / 60)).clip(lower=0)
        temp_df['is_daytime'] = ((temp_df['hour'] >= 6) & (temp_df['hour'] < 18)).astype(int)

        temp_df['peak_sun_factor'] = np.sin(np.pi * (temp_df['hour'] + temp_df['minute']/60 - 6) / 12)
        temp_df['peak_sun_factor'] = temp_df['peak_sun_factor'].clip(lower=0)

        temp_df['hour_sin'] = np.sin(2 * np.pi * temp_df['hour'] / 24)
        temp_df['hour_cos'] = np.cos(2 * np.pi * temp_df['hour'] / 24)
        temp_df['month_sin'] = np.sin(2 * np.pi * temp_df['month'] / 12)
        temp_df['month_cos'] = np.cos(2 * np.pi * temp_df['month'] / 12)

        original_power = temp_df['total_output_power_kW'].copy()

        df_features = temp_df.drop(columns=['total_output_power_kW'])
        df_resampled = df_features.select_dtypes(include=[np.number]).resample(resample_freq).mean()

        df_resampled['historical_power'] = original_power.resample(resample_freq).mean()

        future_target = original_power.shift(-target_horizon)
        df_resampled['power_future'] = future_target.resample(resample_freq).first()

        common_idx = df_resampled.index.intersection(future_target.index)
        df_resampled = df_resampled.loc[common_idx].copy()
        df_resampled['power_future'] = future_target.loc[common_idx]

        for col, ops in self.feature_engineering_plan.items():
            if col not in df_resampled.columns:
                print(f"Warning: {col} not found in dataframe, skipping")
                continue

            if 'lags' in ops:
                for lag in ops['lags']:
                    feature_name = f'{col}_lag_{lag}'
                    df_resampled[feature_name] = df_resampled[col].shift(lag)

            if 'rolling' in ops:
                for method, windows in ops['rolling'].items():
                    for window in windows:
                        feature_name = f'{col}_{method}_{window}'
                        if method == 'mean':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).mean()
                        elif method == 'sum':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).sum()
                        elif method == 'std':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).std()
                        elif method == 'min':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).min()
                        elif method == 'max':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).max()

            if 'diff' in ops:
                for periods in ops['diff']:
                    feature_name = f'{col}_diff_{periods}'
                    df_resampled[feature_name] = df_resampled[col].diff(periods=periods)

        df_resampled.dropna(inplace=True)

        missing_features = [f for f in original_features if f not in df_resampled.columns]
        if missing_features:
            raise ValueError(f"Missing features in prediction data: {missing_features}")

        df_resampled[original_features] = self.scaler.transform(df_resampled[original_features])

        # Create sequences
        X_pred = []
        index = []
        data = df_resampled[original_features].values
        for i in range(len(data) - self.lookback - target_horizon + 1):
            X_pred.append(data[i:i + self.lookback])
            index.append(df_resampled.index[i + self.lookback + target_horizon - 1])  # Align prediction with future

        X_pred = np.array(X_pred)

        if len(X_pred) == 0:
            raise ValueError("Not enough data to generate sequences for prediction.")

        predictions = self.model.predict(X_pred)
        predictions = np.clip(predictions, a_min=0, a_max=None)

        return pd.Series(predictions.flatten(), index=index, name='lstm_predictions')


    def run_pipeline(self, epochs=100, batch_size=64):
        """Run the full pipeline: preprocessing, sequence creation, training, evaluating, and plotting."""
        self.preprocess_data()
        self.create_sequences()
        self.build_model()
        self.train_model(epochs=epochs, batch_size=batch_size)
        results = self.evaluate_model()
        self.plot_results()
        self.plot_training_history()
        return results

# model = LSTMForecastModel(data=df)
# results = model.run_pipeline()
# results

# XGBoost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import seaborn as sns

class ImprovedXGBoostModel:
    def __init__(self, data=None, file_path=None, direct_features=None):
        if data is not None:
            self.df = data.copy()
        elif file_path is not None:
            self.df = pd.read_csv(file_path)
        else:
            raise ValueError("Provide either a DataFrame or file path")

        # Allow user to specify which features to use directly
        #self.direct_features = direct_features or ['WMS 01 irradiancee']

        self.scaler = MinMaxScaler()
        self.model = None

        self.target = None

        # Starting with basic features
        self.features = []

        # Direct features to use (these are passed directly to the model without transformations)
        self.direct_features = ['WMS 01 irradiance','average_temperature_C', 'total_output_power_kW',
            'total_last_active_fault', 'total_net_AC_voltage_V','total_time_run_today_h', 'average_average_cosphii_percent',
       'total_DC_voltage_DCV', 'total_output_current_A',
       'total_grid_connections','total_net_frequency_Hz']

        # Define what features to engineer and how
        self.feature_engineering_plan = {
            'total_output_power_kW': {
                'lags': [1, 2, 3, 6, 12, 24],  # Expanded lag windows
                'rolling': {'mean': [1,3, 6, 12,24], 'std': [3,6], 'min': [3,6], 'max': [1,2,3,6]},
                'diff': [1, 2,3,4,5]  # Rate of change in power
            },
            'average_temperature_C': {
                'lags': [1, 3, 6, 12],
                'rolling': {'mean': [1,2,3, 6,12,24], 'std': [2,3,6]},
                'diff': [1, 2,3]  # Temperature change rate
            },
            'total_DC_voltage_DCV': {
                'lags': [1, 3, 6],
                'rolling': {'mean': [1,2,3, 6,12,24], 'std': [2,3,6]},
                'diff': [1, 2,3]  # Voltage change rate
            },
            'total_output_current_A': {
                'lags': [1, 3, 6],
                'rolling': {'mean': [1,2,3, 6,12,24], 'std': [2,3,6]},
                'diff': [1, 2,3]
            },
            'average_average_cosphii_percent': {
                'lags': [1, 3, 6],
                'rolling': {'mean': [1,2,3, 6,12,24]},
                'diff': [1, 2,3]
            },
            'total_time_run_today_h': {
                'lags': [1, 3, 6],
                'rolling': {'sum': [1,3, 6,12,24]},
                'diff': [1, 2,3]
            },
            'total_last_active_fault': {
                'lags': [1, 3],
                'rolling': {'sum': [1,3, 6,12,24]},
                'diff': [1, 2,3]
            }
        }

        # If solar irradiance is available, add it to plan
        if 'WMS 01 irradiance' in self.df.columns:
            self.feature_engineering_plan['WMS 01 irradiance'] = {
                'lags': [0, 1, 2, 3, 6, 12, 24],
                'rolling': {'mean': [1,3, 6, 12], 'std': [2,3,6], 'max': [1,2, 3,6]},
                'diff': [1, 2, 3,4,5]  # Rate of change in irradiance
            }

    def preprocess_data(self, target_horizon=18, resample_freq='10T'):
        """
        Preprocess data with more sophisticated feature engineering

        Parameters:
        - target_horizon: Number of periods ahead to predict
        - resample_freq: Frequency to resample data ('60T' for hourly)
        """
        # Parse datetime
        if 'datetime' in self.df.columns:
            self.df['datetime'] = pd.to_datetime(self.df['datetime'])
            self.df.set_index('datetime', inplace=True)
        elif 'index' in self.df.columns:
            self.df['datetime'] = pd.to_datetime(self.df['index'])
            self.df.set_index('datetime', inplace=True)
        else:
          # If no datetime column is found, try to infer from 'date' and 'time'
          try:
              self.df['datetime'] = pd.to_datetime(self.df['date'] + ' ' + self.df['time'])
              self.df.set_index('datetime', inplace=True)
          except KeyError:
              raise ValueError("No valid datetime information found. Please provide a 'datetime', 'date' and 'time', or 'index' column.")

        # Enhanced time-based features
        self.df['hour'] = self.df.index.hour
        self.df['day'] = self.df.index.day
        self.df['month'] = self.df.index.month
        self.df['dayofweek'] = self.df.index.dayofweek
        self.df['quarter'] = self.df.index.quarter
        self.df['minute'] = self.df.index.minute

        # Solar position-related features (improved)
        self.df['hours_since_sunrise'] = (self.df['hour'] + self.df['minute'] / 60 - 6).clip(lower=0)
        self.df['hours_until_sunset'] = (18 - (self.df['hour'] + self.df['minute'] / 60)).clip(lower=0)
        self.df['is_daytime'] = ((self.df['hour'] >= 6) & (self.df['hour'] < 18)).astype(int)

        # Peak sun hours approximation (simplified model)
        self.df['peak_sun_factor'] = np.sin(np.pi * (self.df['hour'] + self.df['minute']/60 - 6) / 12)
        self.df['peak_sun_factor'] = self.df['peak_sun_factor'].clip(lower=0)

        # Create cyclic time features to better capture periodicity
        self.df['hour_sin'] = np.sin(2 * np.pi * self.df['hour'] / 24)
        self.df['hour_cos'] = np.cos(2 * np.pi * self.df['hour'] / 24)
        self.df['month_sin'] = np.sin(2 * np.pi * self.df['month'] / 12)
        self.df['month_cos'] = np.cos(2 * np.pi * self.df['month'] / 12)

        # Step 0: Shift irradiance to simulate known-ahead forecast
        self.df['irradiance_future'] = self.df['WMS 01 irradiance'].shift(-target_horizon)

        # Step 1: Keep the original high-frequency target series
        original_target = self.df['total_output_power_kW'].copy()
        future_target = original_target.shift(-target_horizon)
        irradiance_future = self.df['irradiance_future']

        # Step 2: Resample all numeric feature columns except the target
        df_features = self.df.drop(columns=['total_output_power_kW'])
        df_resampled = df_features.select_dtypes(include=[np.number]).resample(resample_freq).mean()

        # Step 3: Resample future target and future irradiance to match resample frequency
        df_resampled['power_future'] = future_target.resample(resample_freq).first()
        df_resampled['irradiance_future'] = irradiance_future.resample(resample_freq).first()

        # Step 4: Align everything using a common index
        common_idx = df_resampled.index.intersection(df_resampled['power_future'].dropna().index)
        df_resampled = df_resampled.loc[common_idx].copy()
        df_resampled['power_future'] = df_resampled['power_future'].loc[common_idx]
        df_resampled['irradiance_future'] = df_resampled['irradiance_future'].loc[common_idx]

        # Feature engineering based on plan
        for col, ops in self.feature_engineering_plan.items():
            if col not in df_resampled.columns:
                print(f"Warning: {col} not found in dataframe, skipping")
                continue

            # Create lag features
            if 'lags' in ops:
                for lag in ops['lags']:
                    feature_name = f'{col}_lag_{lag}'
                    df_resampled[feature_name] = df_resampled[col].shift(lag)
                    self.features.append(feature_name)

            # Create rolling window features
            if 'rolling' in ops:
                for method, windows in ops['rolling'].items():
                    for window in windows:
                        feature_name = f'{col}_{method}_{window}'
                        if method == 'mean':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).mean()
                        elif method == 'sum':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).sum()
                        elif method == 'std':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).std()
                        elif method == 'min':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).min()
                        elif method == 'max':
                            df_resampled[feature_name] = df_resampled[col].rolling(window=window).max()
                        self.features.append(feature_name)

            # Create difference features (rate of change)
            if 'diff' in ops:
                for periods in ops['diff']:
                    feature_name = f'{col}_diff_{periods}'
                    df_resampled[feature_name] = df_resampled[col].diff(periods=periods)
                    self.features.append(feature_name)

        # Add time features to feature list
        time_features = ['hour', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
                         'hours_since_sunrise', 'hours_until_sunset', 'is_daytime',
                         'peak_sun_factor']

        df_resampled[time_features] = self.df[time_features]
        self.features += time_features

        # Create interaction features if solar irradiance is available
        if 'WMS 01 irradiance' in df_resampled.columns:
            df_resampled['irr_temp_interaction'] = df_resampled['WMS 01 irradiance'] * df_resampled['average_temperature_C']
            df_resampled['irr_time_interaction'] = df_resampled['WMS 01 irradiance'] * df_resampled['peak_sun_factor']
            self.features += ['irr_temp_interaction', 'irr_time_interaction']

        # Create target (future power output)
        #df_resampled['power_future'] = df_resampled['total_output_power_kW'].shift(-target_horizon)

        # Drop rows with NaN values
        self.df_clean = df_resampled.dropna()

        # Add for later use in metaModel
        self.target = self.df_clean['power_future']

        # Add direct features if they exist in the dataframe
        for feature in self.direct_features:
            if feature in df_resampled.columns:
                self.features.append(feature)
                print(f"Added direct feature: {feature}")

        # Print final feature list count
        print(f"Total features created: {len(self.features)}")
        print(f"THES ARE THE COLUMNS: {[col for col in self.df_clean.columns]}")

    def analyze_features(self):
        """Analyze feature importance and relationships"""
        X = self.df_clean[self.features]
        y = self.df_clean['power_future']

        # Correlation heatmap of top features with target
        corr = pd.DataFrame(X.corrwith(y).sort_values(ascending=False)).reset_index()
        corr.columns = ['Feature', 'Correlation']

        plt.figure(figsize=(10, 12))
        top_n = min(20, len(corr))
        sns.barplot(x='Correlation', y='Feature', data=corr.head(top_n))
        plt.title(f'Top {top_n} Feature Correlations with Target')
        plt.tight_layout()
        plt.show()

        # Plot relationship between top features and target
        top_features = corr.head(5)['Feature'].tolist()

        fig, axes = plt.subplots(len(top_features), 1, figsize=(12, 15))
        for i, feature in enumerate(top_features):
            axes[i].scatter(X[feature], y, alpha=0.4)
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('Future Power Output')
            axes[i].set_title(f'{feature} vs Target')
            axes[i].grid(True)

        plt.tight_layout()
        plt.show()

        return corr

    def get_important_features(self, top_n=None, threshold=None):
        """
        Get a list of important features based on trained model.

        Parameters:
        - top_n: Select the top N features by importance.
        - threshold: Select features with importance above this threshold (e.g., 0.01).

        Returns:
        - List of selected feature names
        """
        if self.model is None:
            raise ValueError("Model must be trained first to compute feature importance.")

        importance_df = pd.DataFrame({
            'Feature': self.features,
            'Importance': self.model.feature_importances_
        })

        if top_n is not None:
            selected = importance_df.sort_values('Importance', ascending=False).head(top_n)['Feature'].tolist()
        elif threshold is not None:
            selected = importance_df[importance_df['Importance'] >= threshold]['Feature'].tolist()
        else:
            raise ValueError("Provide either top_n or threshold.")

        return selected

    def train_model(self, selected_features=None, param_grid=None):
        """Train XGBoost model with improved grid search"""
        if selected_features:
            used_features = [f for f in selected_features if f in self.features]
        else:
            used_features = self.features

        X = self.df_clean[used_features]
        y = self.df_clean['power_future']

        # Train-test split with stratified temporal split
        split_idx = int(len(self.df_clean) * 0.8)
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

        self.X_train, self.X_test = X_train, X_test
        self.y_train, self.y_test = y_train, y_test

        # Scale features
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)

        # # Default param grid if none provided
        # if param_grid is None:
        #     param_grid = {
        #         'n_estimators': [100, 200, 300],
        #         'max_depth': [5, 7, 9],
        #         'learning_rate': [0.01, 0.05, 0.1],
        #         'subsample': [0.8, 0.9, 1.0],
        #         'colsample_bytree': [0.8, 0.9, 1.0],
        #         'min_child_weight': [1, 3, 5],
        #         'gamma': [0, 0.1, 0.2]
        #     }

        # # Grid search with cross-validation
        # grid = RandomizedSearchCV(
        #     XGBRegressor(objective='reg:squarederror', random_state=42),
        #     param_distributions=param_grid,
        #     n_iter=30,  # Try 20 random combinations
        #     cv=3,
        #     scoring='neg_mean_squared_error',
        #     verbose=1,
        #     n_jobs=-1
        # )

        # grid.fit(self.X_train_scaled, self.y_train)
        # self.model = grid.best_estimator_

        # print(f"Best parameters: {grid.best_params_}")

        default_params = {
            'n_estimators': 300,
            'max_depth': 5,
            'learning_rate': 0.01,
            'subsample': 0.9,
            'colsample_bytree': 0.8,
            'min_child_weight': 1,
            'gamma': 0,
            'objective': 'reg:quantileerror',
            'quantile_alpha': 0.80,
            'random_state': 42
        }

        # Initialize and fit the model
        self.model = XGBRegressor(**default_params)
        self.model.fit(self.X_train_scaled, self.y_train)

        # Get feature importance
        feature_importance = pd.DataFrame({
            'Feature': used_features,
            'Importance': self.model.feature_importances_
        }).sort_values('Importance', ascending=False)

        plt.figure(figsize=(10, 12))
        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
        plt.title('Feature Importance (Top 20)')
        plt.tight_layout()
        plt.show()

        return feature_importance

    def evaluate_model(self):
        """Evaluate model with additional metrics"""
        # Make predictions
        y_pred_train = self.model.predict(self.X_train_scaled)
        y_pred_test = self.model.predict(self.X_test_scaled)

        # Clip negative predictions to zero
        y_pred_train = np.clip(y_pred_train, a_min=0, a_max=None)
        y_pred_test = np.clip(y_pred_test, a_min=0, a_max=None)

        # Calculate metrics
        train_metrics = {
            'rmse': np.sqrt(mean_squared_error(self.y_train, y_pred_train)),
            'mae': mean_absolute_error(self.y_train, y_pred_train),
            'r2': r2_score(self.y_train, y_pred_train)
        }

        test_metrics = {
            'rmse': np.sqrt(mean_squared_error(self.y_test, y_pred_test)),
            'mae': mean_absolute_error(self.y_test, y_pred_test),
            'r2': r2_score(self.y_test, y_pred_test)
        }

        # Calculate normalized metrics
        y_mean = self.y_test.mean()
        test_metrics['nrmse'] = test_metrics['rmse'] / y_mean
        test_metrics['nmae'] = test_metrics['mae'] / y_mean

        print("\nTraining Metrics:")
        for k, v in train_metrics.items():
            print(f"{k.upper()}: {v:.4f}")

        print("\nTest Metrics:")
        for k, v in test_metrics.items():
            print(f"{k.upper()}: {v:.4f}")

        self.metrics = test_metrics
        return train_metrics, test_metrics

    def plot_results(self):
        """Plot comprehensive evaluation results"""
        y_pred = self.model.predict(self.X_test_scaled)
        y_pred = np.clip(y_pred, a_min=0, a_max=None)

        # Create a DataFrame with actual and predicted values
        results_df = pd.DataFrame({
            'Actual': self.y_test.values,
            'Predicted': y_pred
        }, index=self.y_test.index)

        # Plot actual vs predicted time series
        plt.figure(figsize=(15, 6))
        plt.plot(results_df.index, results_df['Actual'], label='Actual', color='blue', alpha=0.7)
        plt.plot(results_df.index, results_df['Predicted'], label='Predicted', color='red', alpha=0.7)
        plt.title('Solar Power Prediction - Actual vs Predicted')
        plt.ylabel('Power Output (kW)')
        plt.xlabel('Time')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot scatter of actual vs predicted
        plt.figure(figsize=(10, 6))
        plt.scatter(results_df['Actual'], results_df['Predicted'], alpha=0.5)

        # Add perfect prediction line
        max_val = max(results_df['Actual'].max(), results_df['Predicted'].max())
        plt.plot([0, max_val], [0, max_val], 'r--')

        plt.title('Actual vs Predicted Power Output')
        plt.xlabel('Actual Power (kW)')
        plt.ylabel('Predicted Power (kW)')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot residuals
        residuals = results_df['Actual'] - results_df['Predicted']
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.scatter(results_df['Predicted'], residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.title('Residuals vs Predicted')
        plt.xlabel('Predicted Power (kW)')
        plt.ylabel('Residual (Actual - Predicted)')
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.hist(residuals, bins=30, alpha=0.7)
        plt.title('Residual Distribution')
        plt.xlabel('Residual Value')
        plt.ylabel('Frequency')
        plt.grid(True)

        plt.tight_layout()
        plt.show()

        # Plot predictions for a sample period (2 weeks)
        if len(results_df) > 14*24:  # If we have at least 2 weeks of data
            sample_period = results_df.iloc[-14*24:]  # Last 2 weeks

            plt.figure(figsize=(15, 6))
            plt.plot(sample_period.index, sample_period['Actual'], label='Actual', color='blue')
            plt.plot(sample_period.index, sample_period['Predicted'], label='Predicted', color='red')
            plt.title('2-Week Sample Period - Actual vs Predicted')
            plt.ylabel('Power Output (kW)')
            plt.xlabel('Time')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()

        # Display error over time of day
        results_df['hour'] = results_df.index.hour
        results_df['abs_error'] = abs(results_df['Actual'] - results_df['Predicted'])

        hour_error = results_df.groupby('hour')['abs_error'].mean()

        plt.figure(figsize=(12, 6))
        hour_error.plot(kind='bar')
        plt.title('Mean Absolute Error by Hour of Day')
        plt.xlabel('Hour of Day')
        plt.ylabel('Mean Absolute Error')
        plt.grid(True, axis='y')
        plt.tight_layout()
        plt.show()

    def predict(self, data=None):
        """
        Predict on the full dataset (no train-test split).
        Useful when using this model as a feature generator for meta-models.

        Parameters:
        - data: optional, custom DataFrame to predict on.
                If None, will use self.df_clean from previous preprocessing.

        Returns:
        - predictions: np.array of predicted values
        - index: index of predictions (datetime or row index)
        """
        # if self.model is None:
        #     raise ValueError("Model has not been trained yet. Call train_model() first.")

        # if data is None:
        #     if not hasattr(self, 'df_clean'):
        #         raise ValueError("No preprocessed data available. Call preprocess_data() first.")
        #     X_full = self.df_clean[self.features]
        #     index = self.df_clean.index
        # else:
        #     # Use passed data (must already have matching features)
        #     missing = [feat for feat in self.features if feat not in data.columns]
        #     if missing:
        #         raise ValueError(f"Missing required features: {missing}")
        if data is None:
            data = self.df_clean

        X_full = data[self.features]
        index = data.index

        X_scaled = self.scaler.transform(X_full)
        predictions = self.model.predict(X_scaled)
        predictions = np.clip(predictions, a_min=0, a_max=None)

        return pd.Series(predictions, index=index, name="xgb_predictions")


    def test_for_leakage(self):
        """Leakage test: Train on shuffled targets and check performance"""

        # Shuffle the target values (break any true pattern)
        y_train_shuffled = np.random.permutation(self.y_train)

        # Train a new XGBoost model on the same features but with shuffled targets
        shuffled_model = XGBRegressor(objective='reg:squarederror', random_state=42)
        shuffled_model.fit(self.X_train_scaled, y_train_shuffled)

        # Predict on the test set
        y_pred_test = shuffled_model.predict(self.X_test_scaled)

        # Evaluate performance on real test targets
        rmse = np.sqrt(mean_squared_error(self.y_test, y_pred_test))
        mae = mean_absolute_error(self.y_test, y_pred_test)
        r2 = r2_score(self.y_test, y_pred_test)

        print("\n=== Leakage Detection Test ===")
        print("Model trained on SHUFFLED target values:")
        print(f"Test RMSE: {rmse:.4f}")
        print(f"Test MAE: {mae:.4f}")
        print(f"Test R²: {r2:.4f}")

        if r2 > 0.2:
            print("⚠️ WARNING: Model is performing too well on shuffled data. Possible leakage!")
        else:
            print("✅ No obvious leakage detected.")

        return {'rmse': rmse, 'mae': mae, 'r2': r2}


    def run_pipeline(self, target_horizon=18, resample_freq='30T'):
        """Run the complete pipeline"""
        print("Step 1: Preprocessing data...")
        self.preprocess_data(target_horizon=target_horizon, resample_freq=resample_freq)

        print("\nStep 2: Analyzing features...")
        feature_corr = self.analyze_features()

        print("\nstep 3: Get top features...")
        model.train_model()
        top_features = model.get_important_features(top_n=30)

        print("\nTop 30 Features Used: ")
        for i, feat in enumerate(top_features, start=1):
            print(f"{i}. {feat}")

        print("\nStep 3: Training model...")
        feature_importance = self.train_model(selected_features = top_features)

        print("\nStep 4: Evaluating model...")
        train_metrics, test_metrics = self.evaluate_model()

        print("\nStep 5: Plotting results...")
        self.plot_results()

        return {
            'feature_correlation': feature_corr,
            'feature_importance': feature_importance,
            'metrics': test_metrics
        }

# Usage example
# if __name__ == "__main__":
    # Example of how to use the model with solar irradiance as a direct feature

    # Option 1: Using default direct features (solar_irradiance)
    # model = ImprovedXGBoostModel(data=df.reset_index())
    # results = model.run_pipeline()
    # model.train_model()
    # model.test_for_leakage()
    # model.test_for_leakage()
    # # Option 2: Specify custom direct features
    # model = ImprovedXGBoostModel(
    #     file_path="your_solar_data.csv",
    #     direct_features=['solar_irradiance']
    # )
    # results = model.run_pipeline()


# Blender

In [ ]:
class blender:
    def __init__(self, data, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
        self.data = data
        self.train_ratio = train_ratio
        self.val_ratio = val_ratio
        self.test_ratio = test_ratio
        self.split_data()

    def split_data(self):
        total_len = len(self.data)
        train_end = int(self.train_ratio * total_len)
        val_end = train_end + int(self.val_ratio * total_len)

        self.base_train_data = self.data.iloc[:train_end]
        self.holdout_data = self.data.iloc[train_end:val_end]
        self.test_data = self.data.iloc[val_end:]

    def run_models(self, lstm_freq='60min', xgb_freq='1H', lstm_horizon=18, xgb_horizon=18):
        # === Train LSTM ===
        self.lstm_model = LSTMForecastModel(data=self.base_train_data)
        self.lstm_model.preprocess_data(resample_freq=lstm_freq, target_horizon=lstm_horizon)
        self.lstm_model.create_sequences()
        self.lstm_model.build_model()
        self.lstm_model.train_model()
        self.lstm_model.evaluate_model()

        # === LSTM Predictions ===
        self.lstm_holdout = self.lstm_model.predict(
            new_data=self.holdout_data,
            resample_freq=lstm_freq,
            target_horizon=lstm_horizon
        )

        self.y_lstm_test = self.lstm_model.predict(
            new_data=self.test_data,
            resample_freq=lstm_freq,
            target_horizon=lstm_horizon
        )

        # === Train XGBoost ===
        self.xgb_model = ImprovedXGBoostModel(data=self.base_train_data.reset_index())
        self.xgb_model.preprocess_data(resample_freq=xgb_freq, target_horizon=xgb_horizon)
        self.xgb_model.train_model()
        self.xgb_model.evaluate_model()

        # === XGBoost Predictions ===
        # Create nwe model for holdout data
        xgb_holdout_model = ImprovedXGBoostModel(data=self.holdout_data.reset_index())
        xgb_holdout_model.preprocess_data(resample_freq=xgb_freq, target_horizon=xgb_horizon)
        #reuse the trained model and scaler
        xgb_holdout_model.model = self.xgb_model.model
        xgb_holdout_model.scaler = self.xgb_model.scaler
        # predictions
        self.xgb_holdout = xgb_holdout_model.predict(data=xgb_holdout_model.df_clean)

        # Create new model for the test data
        xgb_test_model = ImprovedXGBoostModel(data=self.test_data.reset_index())
        xgb_test_model.preprocess_data(resample_freq=xgb_freq, target_horizon=xgb_horizon)
        xgb_test_model.model = self.xgb_model.model
        xgb_test_model.scaler = self.xgb_model.scaler
        # predict
        self.xgb_test = xgb_test_model.predict(data=xgb_test_model.df_clean)

        # === Align predictions and true values on holdout ===
        y_true_holdout = xgb_holdout_model.target
        common_index = y_true_holdout.index.intersection(self.xgb_holdout.index).intersection(self.lstm_holdout.index)

        self.blend_df = pd.DataFrame({
            'y_true': y_true_holdout.loc[common_index],
            'lstm_pred': self.lstm_holdout.loc[common_index].values,
            'xgb_pred': self.xgb_holdout.loc[common_index].values
        }, index=common_index)

        # Align test set predictions for blended model
        y_true_test = xgb_test_model.target
        common_test_index = y_true_test.index.intersection(self.y_lstm_test.index).intersection(self.xgb_test.index)

        X_blend_df = pd.DataFrame({
            'lstm_pred': self.y_lstm_test.loc[common_test_index].values,
            'xgb_pred': self.xgb_test.loc[common_test_index].values
        }, index=common_test_index)

        y_true_aligned = y_true_test.loc[common_test_index]

        self.X_blend_test = X_blend_df
        self.y_blend_true = y_true_aligned

    def train_blended_model(self):
        from sklearn.linear_model import Ridge
        from sklearn.linear_model import RidgeCV
        from sklearn.preprocessing import StandardScaler

        X_blend = self.blend_df[['lstm_pred', 'xgb_pred']]
        y_blend = self.blend_df['y_true']

        # scaler = StandardScaler()
        # X_blend_scaled = scaler.fit_transform(X_blend)

        self.blend_model = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0])
        self.blend_model.fit(X_blend, y_blend)

        # Align final test set inputs
        X_blend_df = self.X_blend_test[['lstm_pred', 'xgb_pred']]
        # X_blend_df_scaled = scaler.transform(X_blend_df)

        self.blend_model_pred = pd.Series(
            self.blend_model.predict(X_blend_df),
            index=X_blend_df.index
        )

        # self.blend_model_pred = self.blend_model_pred[self.X_blend_test.index.get_indexer(features.index)]

        return self.blend_model_pred

    def _generate_time_features(self, index):
        """
        Generates time-based features given a datetime index.

        Parameters:
        - index: pd.DatetimeIndex used to generate features.

        Returns:
        - DataFrame of time features.
        """
        features = pd.DataFrame(index=index)
        features['hour'] = index.hour
        features['minute'] = index.minute
        features['dayofweek'] = index.dayofweek  # 0=Monday, 6=Sunday
        features['is_weekend'] = index.dayofweek >= 5
        features['month'] = index.month
        features['dayofyear'] = index.dayofyear
        features['weekofyear'] = index.isocalendar().week.astype(int)
        features['quarter'] = index.quarter

        return features

    def _generate_residual_features(self, max_lag=5):
        """
        Generate features for residual correction including time-based and lag features.

        Parameters:
        - max_lag: Number of lags to generate for predictions and residuals.

        Returns:
        - DataFrame of features aligned with the holdout set (blend_df).
        """
        df = self.blend_df.copy()

        # Step 1: Time features
        time_features = self._generate_time_features(df.index)

        # Step 2: Lag features for predictions and residuals
        for lag in range(1, max_lag + 1):
            time_features[f'lstm_pred_lag_{lag}'] = df['lstm_pred'].shift(lag)
            time_features[f'xgb_pred_lag_{lag}'] = df['xgb_pred'].shift(lag)

            # Optionally, lag the residuals themselves
            # residuals = df['y_true'] - df[['lstm_pred', 'xgb_pred']].mean(axis=1)
            # time_features[f'residual_lag_{lag}'] = residuals.shift(lag)


        # Drop rows with NaNs from shifting
        time_features.dropna(inplace=True)

        # Also drop rows in blend_df to keep alignment
        self.blend_df = self.blend_df.loc[time_features.index]

        return time_features

    def train_residual_model(self):
        """
        Trains a model to predict the residual error from the meta-model,
        using time-based features.
        """
        from xgboost import XGBRegressor

        # compute residuals
        residual_target = self.blend_df['y_true'] - self.blend_model.predict(self.blend_df[['lstm_pred', 'xgb_pred']])

        # create time-based features with blend_df index
        time_features = self._generate_time_features(self.blend_df.index)
        residual_features = self._generate_residual_features(max_lag=3)

        print(self.holdout_data.columns)
        original_features = self.holdout_data['WMS 01 irradiance']
        original_features = original_features.loc[self.blend_df.index]  # align
        residual_features = pd.concat([residual_features, original_features], axis=1)

        # train residual model
        self.residual_model = XGBRegressor()
        # self.residual_model.fit(time_features, residual_target)
        self.residual_model.fit(residual_features, residual_target.loc[residual_features.index])

        # Save time_features for test correction
        self.residual_train_features = time_features

    def apply_residual_correction(self, max_lag=3):
        """
        Applies residual correction on top of the meta-model predictions,
        using time-based features from the test set.
        """

        df_test = self.X_blend_test.copy()

        # Step 1: generate time-based features
        features = self._generate_time_features(df_test.index)

        # Step 2: add lag features
        for lag in range(1, max_lag + 1):
            features[f'lstm_pred_lag_{lag}'] = df_test['lstm_pred'].shift(lag)
            features[f'xgb_pred_lag_{lag}'] = df_test['xgb_pred'].shift(lag)

        # Step 3: add external features
        extra_features = self.test_data['WMS 01 irradiance']
        extra_features = extra_features.loc[features.index]
        features = pd.concat([features, extra_features], axis=1)

        # Step 4: Drop NaNs from shifting
        features.dropna(inplace=True)
        self.X_blend_test = self.X_blend_test.loc[features.index]
        self.y_blend_true = self.y_blend_true.loc[features.index]
        self.blend_model_pred = self.blend_model_pred[features.index]

        # Step 5: predict residuals & correct
        residual_pred = self.residual_model.predict(features)
        corrected_preds = self.blend_model_pred + residual_pred

        # Step 6: clip negative values
        corrected_preds = np.clip(corrected_preds, 0, None)

        # Step 7: optionally force nighttime values to 0
        # night_mask = self.test_data.loc[corrected_preds.index, 'is_daytime'] == 0
        # corrected_preds[night_mask] = 0

        # Update prediction
        self.blend_model_pred = corrected_preds

        return corrected_preds


    def evaluate_blend_model(self, y_test_true):
        from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

        # align actual with prediction index
        y_true = y_test_true.loc[self.X_blend_test.index]
        y_pred = self.blend_model_pred

        # metrics
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)

        print(f"Meta-Model Evaluation:\nMAE: {mae:.3f} | RMSE: {rmse:.3f} | R²: {r2:.3f}")

        # Plot actual vs predicted
        plt.figure(figsize=(12, 6))
        plt.plot(y_true.index, y_true.values, label='Actual', linewidth=2)
        plt.plot(y_true.index, y_pred, label='Meta Predicted', linestyle='--')
        plt.title("Meta Model: Actual vs Predicted")
        plt.xlabel("Time")
        plt.ylabel("Power Output (kW)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        return mae, rmse, r2

In [ ]:
blender = blender(df)
blender.run_models(lstm_freq='15min', xgb_freq='15min', lstm_horizon=18, xgb_horizon=18)

In [ ]:
blender.train_blended_model()
blender.evaluate_blend_model(blender.y_blend_true)

In [ ]:
blender.train_blended_model()
blender.train_residual_model()
blender.apply_residual_correction()
blender.evaluate_blend_model(blender.y_blend_true)